<a href="https://colab.research.google.com/github/Jaise152/Modern-Natural-Language-Processing-in-Python/blob/main/CNN_for_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install beautifulsoup4

In [1]:
import pandas as pd
import numpy as np
import re
import math
from bs4 import BeautifulSoup
from google.colab import drive 

In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds

In [3]:
drive.mount("/content/drive/")

Mounted at /content/drive/


In [4]:
cols = ['sentiment', 'id', 'date', 'query', 'user','text']

In [5]:
test_data = pd.read_csv(
   "/content/drive/MyDrive/Colab Notebooks/Modern-Natural-Language-Processing-in-Python/test.csv",
    header = None,
    names = cols,
    engine = 'python',
    encoding = 'latin1'
)

In [6]:
train_data = pd.read_csv(
   "/content/drive/MyDrive/Colab Notebooks/Modern-Natural-Language-Processing-in-Python/train.csv",
    header = None,
    names = cols,
    engine = 'python',
    encoding = 'latin1'
)

In [7]:
test_data.tail(3)

,sentiment,id,date,query,user,text
495,4,14074,Sun Jun 14 04:36:34 UTC 2009,latex,iamtheonlyjosie,Ahhh... back in a *real* text editing environm...
496,0,14075,Sun Jun 14 21:36:07 UTC 2009,iran,plutopup7,"Trouble in Iran, I see. Hmm. Iran. Iran so far..."
497,0,14076,Sun Jun 14 21:36:17 UTC 2009,iran,captain_pete,Reading the tweets coming out of Iran... The w...


In [8]:
test_data.drop(['id','date','query','user'],
          axis = 1,
          inplace = True)

In [9]:
train_data.drop(['id','date','query','user'],
          axis = 1,
          inplace = True)

In [10]:
len(train_data)

1600000

In [11]:
BeautifulSoup(test_data['text'][0], 'lxml',).get_text()

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [12]:
re.sub(r"@[A-Z a-z 0-9]+", ' ', test_data['text'][0])

' . Not that the DX is cool, but the 2 is fantastic in its own right.'

In [13]:
re.sub(r"https?://[A-Za-z0-9./]+", ' ', test_data['text'][0])

'@stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool, but the 2 is fantastic in its own right.'

In [14]:
re.sub(r"[^A-Za-z0-9.!?]", ' ', test_data['text'][0])

' stellargirl I loooooooovvvvvveee my Kindle2. Not that the DX is cool  but the 2 is fantastic in its own right.'

In [15]:
def clean_tweet(tweet):
  tweet = re.sub(r"@[A-Z a-z 0-9]+", ' ', tweet) #Removing mentiones
  tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet) #Removing urls
  tweet = re.sub(r"[^A-Za-z0-9.!?]+", ' ', tweet) #Removing all unwanted characters
  tweet = re.sub(r" +", ' ', tweet)
  return tweet

In [16]:
#clean_tweet_test = [clean_tweet(tweet) for tweet in train_data.text ]

In [17]:
clean_tweet_train = [clean_tweet(tweet) for tweet in train_data.text ]

In [18]:
len(clean_tweet_train)

1600000

In [19]:
train_data['sentiment'].unique()

array([0, 4])

In [20]:
train_data['sentiment'].unique()

array([0, 4])

In [21]:
data_labels = train_data.sentiment.values

In [22]:
data_labels

array([0, 0, 0, ..., 4, 4, 4])

In [23]:
data_labels[data_labels==4] =1

In [24]:
set(data_labels)

{0, 1}

Tokenization

In [25]:
#
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    clean_tweet_train, target_vocab_size = 2**16
)

In [26]:
clean_tweet_train_input = [tokenizer.encode(sentence) for sentence in clean_tweet_train]

In [27]:
len(clean_tweet_train_input)

1600000

Padding

In [28]:
Max_len = max([len(sentence) for sentence in clean_tweet_train_input ])

In [29]:
clean_tweet_train_input = tf.keras.preprocessing.sequence.pad_sequences(
    clean_tweet_train_input,
    maxlen = Max_len,
    value = 0,
    padding = 'post'
)

In [30]:
len(clean_tweet_train_input)

1600000

In [31]:
test_idx = np.random.randint(0,16000,1600)
test_idx = np.concatenate((test_idx, test_idx+16000))

In [32]:
test_inputs = clean_tweet_train_input[test_idx]
test_labels = data_labels[test_idx]
train_inputs = np.delete(clean_tweet_train_input, test_idx, axis=0)
train_labels = np.delete(data_labels, test_idx)

Model Building

In [44]:
class DCNN(tf.keras.Model):

  def __int__(self,
              vocab_size,
              emb_dim = 128,
              nb_filters = 50,
              FNN_units = 512,
              nb_classes = 2,
              dropout_rate = 0.1,
              training = False,
              name = 'dcnn'):
    super(DCNN, self).__int__(name=name)

    self.embedding = layers.Embedding(vocab_size,
                                      emb_dim)
    self.bigram = layers.Conv1D(filter = nb_filters,
                               kernal_size = 2,
                               padding = "valid",
                               activation = "relu")
    self.pool_1 = layers.GlobalMaxPool1D()

    self.trigram = layers.Conv1D(filter = nb_filters,
                               kernal_size = 3,
                               padding = "valid",
                               activation = "relu")
    self.pool_2 = layers.GlobalMaxPool1D()

    self.fourgram = layers.Conv1D(filter = nb_filters,
                               kernal_size = 4,
                               padding = "valid",
                               activation = "relu")
    self.pool_3 = layers.GlobalMaxPool1D()

    self.dense_1 = layers.Dense(units = FNN_units, activation = "relu")
    self.dropout = layers.Dropout(rate = dropout_rate)

    if nb_classes == 2:
      self.last_dense = layers.Desne(units =1, 
                                     activation = 'sigmoid')
    else:
      self.last_dense = layers.Dense(units = nb_classes,
                                    activation = "relu")
  def call(self, inputs, training):
    x = self.embedding(inputs)
    x_1 = self.bigram(x)
    x_1 = self.pool_1(x_1)
    x_2 = self.trigram(x)
    x_2 = self.pool_2(x_2)
    x_3 = self.fourgram(x)
    x_3 = self.pool_3(x_3)

    merged = tf.concat([x_1, x_2, x_3], axis =1 )

